In [94]:
import os
import json
import pandas as pd
import traceback 
from dotenv import load_dotenv
load_dotenv()
from langchain_huggingface import ChatHuggingFace
import sys
print(sys.executable)
import PyPDF2
from langchain_core.callbacks import BaseCallbackHandler



c:\Users\yadav\mcqgen\.venv\Scripts\python.exe


In [95]:
from langchain_huggingface import HuggingFaceEndpoint

# Get HF token from environment
huggingface_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    huggingfacehub_api_token=huggingface_token,
    temperature=0.5,
    max_new_tokens=512
)

In [96]:
chat_model = ChatHuggingFace(llm=llm)

In [97]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [98]:
RESPONSE_JSON = {
  "1": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here"
    },
    "correct": "correct answer"
  },
    "2": {
        "mcq": "multiple choice question",
        "options": {
        "a": "choice here",
        "b": "choice here",
        "c": "choice here",
        "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
        "a": "choice here",
        "b": "choice here",
        "c": "choice here",
        "d": "choice here"
        },
        "correct": "correct answer"
    },
}


In [99]:
TEMPLATE = """You are an expert MCQ maker.

Text: {text}

Subject: {subject}
Number of Questions: {number}
Tone: {tone}

Create {number} MCQs in {tone} tone. Format: {response_json}

Ensure to make {number} mcqs.You have to generate a quiz in RESPONSE_JSON format only.

"""

In [100]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "subject", "number", "tone", "response_json"],
    template=TEMPLATE
)

In [101]:
quiz_chain1 = quiz_generation_prompt | chat_model | StrOutputParser()

In [102]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.
If the quiz is not at per with the cognitive and analytical abilities of the students,
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students.

Quiz_MCs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [103]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE
)

In [104]:
quiz_chain2 = quiz_evaluation_prompt | chat_model | StrOutputParser()

In [105]:
combined_chain = (
    RunnablePassthrough()
    .assign(quiz=quiz_chain1)
    | quiz_chain2
)

In [106]:
file_path="C:\\Users\\yadav\\mcqgen\\data.txt"

In [107]:
with open(file_path, "r") as f:
    content = f.read()
print(content)

Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]

High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being c

In [108]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [122]:
NUMBER = 7
SUBJECT = "Artificial Intelligence"
TONE = "Easy"

In [123]:
response = combined_chain.invoke(
    {
        "text": content,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)


In [124]:
print(response)

{"1": {"mcq": "What is the primary goal of Artificial Intelligence (AI)?", "options": {"a": "To create machines that can think and learn like humans", "b": "To develop advanced web search engines", "c": "To build autonomous vehicles", "d": "To create virtual assistants"}, "correct": "a"},

"2": {"mcq": "What is the name of the architecture that accelerated the growth of AI after 2017?", "options": {"a": "Neural Network", "b": "Transformer Architecture", "c": "Deep Learning", "d": "Machine Learning"}, "correct": "b"},

"3": {"mcq": "What is the term for AI that can complete virtually any cognitive task at least as well as a human?", "options": {"a": "Artificial General Intelligence (AGI)", "b": "Artificial Narrow Intelligence (ANI)", "c": "Artificial Superintelligence (ASI)", "d": "Artificial Intelligence (AI)"}, "correct": "a"},

"4": {"mcq": "When was Artificial Intelligence founded as an academic discipline?", "options": {"a": "1950", "b": "1956", "c": "1960", "d": "1970"}, "correct"

In [126]:
Quiz = json.loads(response)

In [127]:
quiz_table_data = []
for key, value in Quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )

    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [128]:
quiz_table_data

[{'MCQ': 'What is the primary goal of Artificial Intelligence (AI)?',
  'Choices': 'a: To create machines that can think and learn like humans | b: To develop advanced web search engines | c: To build autonomous vehicles | d: To create virtual assistants',
  'Correct': 'a'},
 {'MCQ': 'What is the name of the architecture that accelerated the growth of AI after 2017?',
  'Choices': 'a: Neural Network | b: Transformer Architecture | c: Deep Learning | d: Machine Learning',
  'Correct': 'b'},
 {'MCQ': 'What is the term for AI that can complete virtually any cognitive task at least as well as a human?',
  'Choices': 'a: Artificial General Intelligence (AGI) | b: Artificial Narrow Intelligence (ANI) | c: Artificial Superintelligence (ASI) | d: Artificial Intelligence (AI)',
  'Correct': 'a'},
 {'MCQ': 'When was Artificial Intelligence founded as an academic discipline?',
  'Choices': 'a: 1950 | b: 1956 | c: 1960 | d: 1970',
  'Correct': 'b'},
 {'MCQ': 'What is the term for the period of rap

In [129]:
Mcq_quiz = pd.DataFrame(quiz_table_data)

In [130]:
Mcq_quiz.to_csv("Machinelearning_MCQ_Easy.csv", index=False)